In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load your data into a pandas DataFrame
data = pd.read_csv('../data/balanced_classification_dataset.csv')  # replace 'your_dataset.csv' with your actual file

In [5]:
data = data.rename(columns={'data': 'text'})
data.head()

,Unnamed: 0.1,Unnamed: 0,patientid,sex,age,ANA,EXA,DIA_text,DIA_code,OP_text,OP_code,label,text
0,0,1,61.0,male,32.0,Beschwerden re. Achillessehne. NMR unauffällig...,Re. Achillessehne: Verdickung im Bereich der A...,Retropatellare Chondromalazie li; Insertionste...,M22.4,NaN,NaN,other,Beschwerden re. Achillessehne. NMR unauffällig...
1,1,2,67.0,male,41.0,Seit mehreren Monaten Beschwerden re. Achilles...,Re. Achillessehne: Deutliche Auftreibung der A...,V.a.laterale Chondromalazie bei st.n. Außenmen...,M23.3,NaN,NaN,meniskus_urgent,Seit mehreren Monaten Beschwerden re. Achilles...
2,2,6,72.0,male,19.0,. .Gestern beim Training stechende Schmerzen l...,Klinischer Befund des Oberschenkels li.deutl.V...,Z.n.arthroskopischer Chirurgie des Kniegelenkes.,M23.3,NaN,NaN,meniskus_urgent,. .Gestern beim Training stechende Schmerzen l...
3,3,7,72.0,male,23.0,beide Knie Schmerzen bei Treppe steigen oder L...,bd. Knie: frei Bewegl.; Patella o.B.; keine MZ...,Z.n.IM-Teilresektion re.,S83.2,NaN,NaN,meniskus_urgent,beide Knie Schmerzen bei Treppe steigen oder L...
4,4,9,84.0,male,57.0,(08.19 Uhr); bewegungsabhängig.Schmerzen wiede...,Klinischer Befund des Kniegelenks li.: Drucksc...,Beginnende Gonarthrose bds. li. mehr als re.,M17.9,NaN,NaN,other,(08.19 Uhr); bewegungsabhängig.Schmerzen wiede...


In [6]:
# Assuming you have two columns: 'text' and 'label'
texts = data['text'].values
labels = data['label'].values

# Encode the labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split the data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, encoded_labels, test_size=0.2, random_state=42
)

In [9]:
from transformers import BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-german-cased')

# Tokenize the data
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=128)

In [10]:
import tensorflow as tf

def create_dataset(encodings, labels):
    dataset = tf.data.Dataset.from_tensor_slices((
        dict(encodings),
        labels
    ))
    return dataset

train_dataset = create_dataset(train_encodings, train_labels)
test_dataset = create_dataset(test_encodings, test_labels)

2024-07-18 15:14:21.162812: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2024-07-18 15:14:21.162838: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2024-07-18 15:14:21.162842: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2024-07-18 15:14:21.162875: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-18 15:14:21.162892: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
from transformers import TFBertForSequenceClassification

# Load the BERT model for sequence classification with the number of output labels
model = TFBertForSequenceClassification.from_pretrained('google-bert/bert-base-german-cased', num_labels=len(label_encoder.classes_))

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [14]:
# Train the model
model.fit(train_dataset.shuffle(1000).batch(32),
          epochs=3,
          batch_size=32)


Epoch 1/3
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2024-07-18 15:15:54.492676: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-07-18 15:15:55.417337: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp_10.


1132/1132 [==============================] - 826s 704ms/step - loss: 1.0926 - accuracy: 0.5533
Epoch 2/3
1132/1132 [==============================] - 727s 642ms/step - loss: 0.9780 - accuracy: 0.6183
Epoch 3/3
1132/1132 [==============================] - 618s 546ms/step - loss: 0.9005 - accuracy: 0.6550


In [15]:
# Evaluate the model
loss, accuracy = model.evaluate(test_dataset.batch(32))
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

283/283 [==============================] - 58s 185ms/step - loss: 1.0327 - accuracy: 0.6021
Loss: 1.0326831340789795
Accuracy: 0.6020768880844116


In [17]:
import numpy as np
predictions = model.predict(test_dataset.batch(32))
predicted_labels = np.argmax(predictions.logits, axis=-1)

# Decode class labels to original labels
decoded_predictions = label_encoder.inverse_transform(predicted_labels)

# True labels from the test set
true_labels = label_encoder.inverse_transform(test_labels)

# Create a DataFrame to compare
comparison_df = pd.DataFrame({'Text': test_texts, 'True Label': true_labels, 'Predicted Label': decoded_predictions})

148/283 [==============>...............] - ETA: 21s

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

cm = confusion_matrix(true_labels, decoded_predictions, labels=label_encoder.classes_)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [16]:
model.save_pretrained('german_bert_classify_model')  # replace with your desired path
tokenizer.save_pretrained('german_bert_classify_tokenizer')  # replace with your desired path

('german_bert_classify_tokenizer/tokenizer_config.json',
 'german_bert_classify_tokenizer/special_tokens_map.json',
 'german_bert_classify_tokenizer/vocab.txt',
 'german_bert_classify_tokenizer/added_tokens.json')